# Gapminder

<div name="html-admonition" style="font-size: 0.8em">
<input type="button" onclick="location.href='https://translate.google.com/translate?hl=&sl=ja&tl=en&u='+window.location;" value="Google translation" style="color:#ffffff;background-color:#008080; height:25px" onmouseover="this.style.background='#99ccff'" onmouseout="this.style.background='#008080'"/> in English or the language of your choice.
</div><br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from gapminder import gapminder
from see import see

[Gapminder](https://www.gapminder.org)とは世界規模で見た経済格差をデータで探る有名なサイトであり、一見の価値があるサイトである。そのサイトで使われているデータを整理してパッケージにまとめたのが`gapminder`である。

````{note}
MacではTerminal、WindowsではGit Bashを使い、次のコマンドで`gapminder`をインストールできる。
```
pip install gapminder
```
````

ここでは`gapminder`に含まれるデータを使い`pandas`の`groupby`という`DataFrame`のメソッドの使い方の例を紹介する。両方ともデータをグループ化して扱う場合に非常に重宝するので、覚えておいて損はしないだろう。

## データ

＜列ラベル＞
* `country`：国名
* `continent`：大陸
* `year`：年
* `lifeExp`：平均寿命
* `pop`：人口
* `gdpPercap`：一人当たりGDP（国内総生産）

データの読み込みと最初の5行の表示

In [ ]:
df = gapminder
df.head()

最後の5行の表示

In [ ]:
df.tail()

データセットの内容確認

In [ ]:
df.info()

記述統計の表示

In [ ]:
df.describe()

含まれる国名の表示

In [ ]:
countries = df.loc[:,'country'].unique()
countries

国の数の確認

In [ ]:
len(countries)

`2007`年における`continent`の内訳（国の数）

In [ ]:
cond = ( df['year']==df['year'].max() )
df.loc[cond,:].value_counts('continent')

割合で表すと

In [ ]:
cond = ( df['year']==df['year'].max() )
df.loc[cond,:].value_counts('continent', normalize=True)

## groupby()

In [ ]:
df_group = df.groupby('continent')

In [ ]:
see(df_group)

### continentの内訳（again）

In [ ]:
country_names = df_group['country'].nunique()
country_names

### 統計量

In [ ]:
three_vars=['lifeExp','pop','gdpPercap']

#### 観測値の数

大陸別の観測値の数

In [ ]:
df_group.size()

棒グラフ

In [ ]:
ax = df_group.size().plot(kind='bar')
ax.set_title('大陸別の観測値の数', size=15)
pass

変数別での観測値の数

In [ ]:
df_group.count()

#### 平均

それぞれの変数の平均（文字列の`country`を含めるとエラーが発生するため`three_vars`を指定する。）

In [ ]:
df_group[three_vars].mean()

`gdpPercap`と`lifeExp`の大陸別平均の散布図

In [ ]:
ax = df_group[three_vars].mean().plot(kind='scatter', x='gdpPercap', y='lifeExp')
ax.set_title('gdpPercapとlifeExpの\n大陸別平均の散布図', size=20)
pass

#### 標準偏差

それぞれの変数の標準偏差

In [ ]:
df_group[three_vars].std()

#### 最大値

In [ ]:
df_group.max()

#### 最小値

In [ ]:
df_group.min()

#### 3変数の記述統計

In [ ]:
df_group[three_vars].describe().applymap("{0:.1f}".format).T

### groupby.agg()

`agg()`を使うとよりメソッドだけではなく，他の関数も使える。

`()`の中に関数を入れる。

In [ ]:
df_group[three_vars].agg(np.mean)

In [ ]:
df_group[three_vars].agg([np.max, np.min, np.mean])

In [ ]:
# 自作の関数もOK

func = lambda x: ( np.max(x)-np.min(x) )/np.mean(x)

df_group[['lifeExp','pop','gdpPercap']].agg(func)

`continent`の内訳の割合を計算

### 図

**`continent`平均寿命**

In [ ]:
df_lifeExp_continent = df_group['lifeExp'].mean()

In [ ]:
ax = df_lifeExp_continent.plot(kind='bar')
ax.set_title('大陸別平均寿命', size=15)
pass

#### 3つの変数

In [ ]:
df_mean = df_group[three_vars].mean()
df_mean['ln_pop'] = np.log( df_mean['pop'] )
df_mean['ln_gdpPercap'] = df_mean['gdpPercap'].apply(np.log)
df_mean['lifeExp_10'] = df_mean['lifeExp']/10

In [ ]:
df_mean[['ln_pop','lifeExp_10', 'ln_gdpPercap']].plot(kind='bar')
pass

### 複数階層の`groupby()`

`continent`別の平均時系列を考えるときに有用。

In [ ]:
df_group2 = df.groupby(['continent','year'])

In [ ]:
df_group2[three_vars].mean().head()

In [ ]:
# lifeExpの列だけを選択した後，unstackを使ってyearが行ラベルになるDataFrameに変換

df_lifeExp_group = df_group2[three_vars].mean().loc[:,'lifeExp'].unstack(level=0)

In [ ]:
ax = df_lifeExp_group.plot()
ax.set_title('大陸別平均寿命の推移', size=15)
pass

**世界平均との比較**

In [ ]:
df_group_year = df.groupby('year')

In [ ]:
world_lifeExp = df_group_year[three_vars].mean()['lifeExp'].to_numpy().reshape(1,12).T

In [ ]:
df_lifeExp_diff = df_lifeExp_group - world_lifeExp

ax = df_lifeExp_diff.plot()
ax.set_title('大陸別平均寿命の世界平均との差の推移', size=15)
pass